In [ ]:
# TODO: Check wind measurements between providers (Matthew says they have different conventions)
# TODO: Make a correlation matrix (consider making with and without zero-speed observations)
# TODO: check correlation between shaft power and engine load (should be above 0.95)
# TODO: Scatterplot for everything that is measured by both providers
# TODO: Scatterplot between Propeller shaft rotational speed and Main Engine Rotational Speed